In [1]:
import os
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
# CUDA配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
# 数据加载
input_data = joblib.load('pre-data/Re.joblib')
true_output_u = joblib.load('pre-data/u0_all.joblib')
true_output_v = joblib.load('pre-data/u1_all.joblib')
true_output_p = joblib.load('pre-data/p_all.joblib')
true_output_Fv0 = joblib.load('pre-data/Fv0_all.joblib')
true_output_Fv1 = joblib.load('pre-data/Fv1_all.joblib')
true_output_Fi0 = joblib.load('pre-data/Fi0_all.joblib')
true_output_Fi1 = joblib.load('pre-data/Fi1_all.joblib')

In [15]:
class ResNet2D(nn.Module):
    def __init__(self, full_connect_shape, q, N):
        super(ResNet2D, self).__init__()
        self.q = q
        self.N = N
        self.C, self.H, self.W, _ = full_connect_shape  # 重新确认维度
        
        # 确保每个全连接层的维度不会过大
        self.fc = nn.Linear(np.prod((1,)), 128)  # 简化fc输出维度，避免过大

        # 基本块（BB）和子块（SB）
        self.bb_layers = nn.ModuleList()
        for _ in range(q):
            sb_layers = nn.ModuleList()
            for _ in range(N):
                sb_layers.append(nn.Conv2d(self.C, self.C, kernel_size=(3, 3), padding=1))
                sb_layers.append(nn.LeakyReLU(0.01))
            self.bb_layers.append(sb_layers)
        
        # 输出层
        self.final_conv_u = nn.Conv2d(self.C, 1, kernel_size=(3, 3), padding=1)
        self.final_conv_v = nn.Conv2d(self.C, 1, kernel_size=(3, 3), padding=1)
        self.final_conv_p = nn.Conv2d(self.C, 1, kernel_size=(3, 3), padding=1)

        # 定义 Fv0, Fv1, Fi0, Fi1 的卷积层
        self.FvLayer = nn.Conv2d(self.C, 1, kernel_size=(3, 3), padding=1, bias=False)
        self.FixLayer = nn.Conv2d(self.C, 1, kernel_size=(1, 2), padding=0, bias=False)
        self.FiyLayer = nn.Conv2d(self.C, 1, kernel_size=(2, 1), padding=0, bias=False)

        # 定义权重
        self.init_weights()

    def init_weights(self):
        with torch.no_grad():
            self.FvLayer.weight = nn.Parameter(torch.tensor([[0, 1, 0], [1, -4, 1], [0, 1, 0]], dtype=torch.float32).reshape(1, 1, 3, 3))
            self.FixLayer.weight = nn.Parameter(torch.tensor([[-1, 1]], dtype=torch.float32).reshape(1, 1, 1, 2))
            self.FiyLayer.weight = nn.Parameter(torch.tensor([[-1], [1]], dtype=torch.float32).reshape(1, 1, 2, 1))

    def forward(self, x):
        # 全连接层
        x = self.fc(x)
        
        # 打印形状，确认大小
        print(f"Shape before reshaping: {x.shape}")
        
        # 动态计算适合的形状
        x = x.view(-1, self.C, self.H, self.W)  # 视图调整
    
        # 基本块
        for sb_layers in self.bb_layers:
            bb_input = x
            for layer in sb_layers:
                x = layer(x)
            x = (bb_input + x) * 2
    
        # 输出计算
        xu = self.final_conv_u(x)
        xv = self.final_conv_v(x)
        xp = self.final_conv_p(x)
    
        Fv0 = self.FvLayer(xu)
        Fv1 = self.FvLayer(xv)
        ux = self.FixLayer(xu)
        uy = self.FiyLayer(xu)
        vx = self.FixLayer(xv)
        vy = self.FiyLayer(xv)
        
        Fi0 = (xu[:, :-1, :-1, :] * ux[:, :-1, :, :]) + (xv[:, :-1, :-1, :] * uy[:, :, :-1, :])
        Fi1 = (xu[:, :-1, :-1, :] * vx[:, :-1, :, :]) + (xv[:, :-1, :-1, :] * vy[:, :, :-1, :])
        
        return xu, xv, xp, Fv0, Fv1, Fi0, Fi1

In [16]:
# 加载已有模型
model_path = 'model/model.pth'
model = ResNet2D(full_connect_shape=(true_output_u.shape[1], true_output_u.shape[2], 1, 128), q=2, N=3).to(device)
if os.path.exists(model_path):
    print(f"Loading existing model from {model_path}")
    model.load_state_dict(torch.load(model_path))
else:
    print("No existing model found, starting with a new model.")

No existing model found, starting with a new model.


In [17]:
# 定义损失函数和优化器
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.L1Loss()

In [18]:
# 训练步骤
def train_step(input_data, true_u, true_v, true_p, true_Fv0, true_Fv1, true_Fi0, true_Fi1, optimizer):
    model.train()
    optimizer.zero_grad()
    
    # 转换为张量并送入设备
    input_data = torch.tensor(input_data, dtype=torch.float32).to(device)
    true_u = torch.tensor(true_u, dtype=torch.float32).to(device)
    true_v = torch.tensor(true_v, dtype=torch.float32).to(device)
    true_p = torch.tensor(true_p, dtype=torch.float32).to(device)
    true_Fv0 = torch.tensor(true_Fv0, dtype=torch.float32).to(device)
    true_Fv1 = torch.tensor(true_Fv1, dtype=torch.float32).to(device)
    true_Fi0 = torch.tensor(true_Fi0, dtype=torch.float32).to(device)
    true_Fi1 = torch.tensor(true_Fi1, dtype=torch.float32).to(device)
    
    # 获取模型输出
    pred_u, pred_v, pred_p, pred_Fv0, pred_Fv1, pred_Fi0, pred_Fi1 = model(input_data)
    
    # 计算损失
    loss_u = loss_fn(pred_u, true_u)
    loss_v = loss_fn(pred_v, true_v)
    loss_p = loss_fn(pred_p, true_p)
    loss_Fv0 = loss_fn(pred_Fv0, true_Fv0)
    loss_Fv1 = loss_fn(pred_Fv1, true_Fv1)
    loss_Fi0 = loss_fn(pred_Fi0, true_Fi0)
    loss_Fi1 = loss_fn(pred_Fi1, true_Fi1)
    
    # 总损失
    loss = loss_u + loss_v + loss_p + loss_Fv0 + loss_Fv1 + loss_Fi0 + loss_Fi1
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [19]:
# 训练模型
def train_model(input_data, true_u, true_v, true_p, true_Fv0, true_Fv1, true_Fi0, true_Fi1, batch_size, epochs, optimizer):
    loss_record = []
    for epoch in range(epochs):
        # 打乱数据
        num_samples = len(input_data)
        indices = np.random.permutation(num_samples)
        
        # 批处理
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_input_data = input_data[batch_indices]
            batch_true_u = true_u[batch_indices]
            batch_true_v = true_v[batch_indices]
            batch_true_p = true_p[batch_indices]
            batch_true_Fv0 = true_Fv0[batch_indices]
            batch_true_Fv1 = true_Fv1[batch_indices]
            batch_true_Fi0 = true_Fi0[batch_indices]
            batch_true_Fi1 = true_Fi1[batch_indices]

            # 调用训练步骤
            loss = train_step(batch_input_data, batch_true_u, batch_true_v, batch_true_p, batch_true_Fv0, batch_true_Fv1, batch_true_Fi0, batch_true_Fi1, optimizer)
            loss_record.append(loss)

        print(f"Epoch {epoch+1}/{epochs} - Loss: {loss}")
    
    return loss_record


In [20]:
loss_info = train_model(input_data, true_output_u, true_output_v, true_output_p, true_output_Fv0, true_output_Fv1, true_output_Fi0, true_output_Fi1, batch_size=10, epochs=2000, optimizer=optimizer)

Shape before reshaping: torch.Size([10, 128])


RuntimeError: shape '[-1, 64, 64, 1]' is invalid for input of size 1280